In [ ]:
import pandas as pd
import numpy as np
import time
import json
from glob import glob
from dateutil.parser import parse

# 合併多個csv檔 - glob

In [ ]:
files = glob('./file*')
print(files)

In [ ]:
df = pd.concat((pd.read_csv(file, dtype={'mls': int}) for file in files), axis='rows', ignore_index=True)

In [ ]:
df.drop_duplicates(subset ="address",keep = False, inplace = True)

In [ ]:
def OutputCSV():      
    Result = "filename_%s.csv" % (str(time.strftime("%Y-%m-%d")))
    df_SAMPLE = pd.DataFrame.from_dict(df)
    df_SAMPLE.to_csv(Result, index=False) 
    print('Finish')

In [ ]:
OutputCSV()

# 合併DataFrame - merge

In [ ]:
df = pd.read_csv('inf_list1.csv')

In [ ]:
df1 = pd.read_csv('years_price_list.csv')

In [ ]:
df2 = df.merge(df1)

In [ ]:
df2

In [ ]:
def OutputCSV1():      
    Result = "inf_list%s.csv" % (str(time.strftime("%Y-%m-%d")))
    df_SAMPLE = pd.DataFrame.from_dict(df2)
    df_SAMPLE.to_csv(Result, index=False) 
    print('Finish')

In [ ]:
OutputCSV1()

# 年與房價資料

In [ ]:
df = pd.read_csv('exchange_rate.csv')

In [ ]:
cols = ['zipcode','sqft','soldout','houseprice']

In [ ]:
df1 = df[cols]

In [ ]:
df1

In [ ]:
for i in range(len(df1)):
    df1['soldout'][i] = parse(df1['soldout'][i]).year

In [ ]:
df1

In [ ]:
years_price_list = []
years = [2017,2018,2019,2020] # ,2019,2018,2017
counts = 0
for i in [98010, 98029, 98075 ,98092,98106]:
    years_price = [i]
    for j in years:
        if j == 2017:
            df2 = df1[(df1['zipcode']==i)&(df1['soldout']>(j-19))&(df1['soldout']<(j+2))] 
        else:
            df2 = df1[(df1['zipcode']==i)&(df1['soldout']>(j))&(df1['soldout']<(j+2))] 
        df3 = df2.groupby('zipcode').apply(sum)
        years_price += [round(df3.iloc[0,3]/df3.iloc[0,1],2)]
    years_price_list.append(years_price)
    years_price = []
    counts += 1

In [ ]:
df3

In [ ]:
years_price_list

In [ ]:
def OutputCSV2():
    cols = ['zipcode','before2018','2019y','2020y','2021y']
    file_name = "years_price_list.csv"
    pd.DataFrame(years_price_list, columns = cols).to_csv(file_name, index = False, encoding = "UTF-8")

In [ ]:
OutputCSV2()

# 測試轉JSON

In [ ]:
df = pd.read_csv('exchange_rate.csv')

In [ ]:
cols = ['zipcode','walkscore','transitscore','grade1_5','grade6_8','grade9_12']

In [ ]:
df1 = df[cols]

In [ ]:
df1 = df1.head(5)

In [ ]:
df1

In [ ]:
df1.to_json(orient="index",force_ascii=False)

In [ ]:
type(df1)

In [ ]:
pd.DataFrame(df1).to_json('json_test.json')

### json測試

In [ ]:
df2 = eval(df1)

In [ ]:
type(df2)

In [ ]:
df2 = str(df1)

In [ ]:
df2

In [ ]:
for i in range(10):
    df2 = df2.replace(f'"{i}":','') 
                    # (f'"{i}":','') #(f"'{i}':",'')

In [ ]:
df2

In [ ]:
df2 = df2.replace('{{','[{').replace('}}','}]')

In [ ]:
df2

In [ ]:
df2 = df2.replace('"',"'")

In [ ]:
df2

In [ ]:
import json
with open('data.json', 'w') as f:
  json.dump(df1, f, ensure_ascii=False)

# 建立比較資料

In [ ]:
df = pd.read_csv('exchange_rate.csv')

In [ ]:
df.shape

In [ ]:
cols = ['zipcode','walkscore','transitscore','grade1_5','grade6_8','grade9_12',
          'WhiteRatio','AfricanRatio','LatinoRatio','AsianRatio','IndianRatio','OthersRatio']

In [ ]:
df1 = df[cols]

In [ ]:
df1

In [ ]:
dfzipcode = df['zipcode'].drop_duplicates()

In [ ]:
zipcodelist = []
for j in range(len(dfzipcode)):
    zipcodelist += [dfzipcode.iloc[j]]
zipcodelist = sorted(zipcodelist)

In [ ]:
zipcodelist

In [ ]:
inf_list = []
counts = 0
df2 = df1.groupby('zipcode').apply(sum)
len1 = len(df2.iloc[0])
for i in zipcodelist:
    df3 = df2.iloc[counts]/len(df1[(df1['zipcode']==i)])
    inf_list.append([int(df3[len1-12]),round(df3[len1-11],2),round(df3[len1-10],2),round(df3[len1-9],2),round(df3[len1-8],2),round(df3[len1-7],2),round(df3[len1-6],2),
                     round(df3[len1-5],2),round(df3[len1-4],2),round(df3[len1-3],2),round(df3[len1-2],2),round(df3[len1-1],2)])
    counts+=1

In [ ]:
inf_list1 = []
for i in range(len(inf_list)):
    if inf_list[i][0] in [98010, 98029, 98092, 98075 ,98106]:
        inf_list1.append(inf_list[i])
print(inf_list1)

In [ ]:
def OutputCSV3():
    file_name = "inf_list1.csv"
    pd.DataFrame(inf_list1, columns = cols).to_csv(file_name, index = False, encoding = "UTF-8")

In [ ]:
OutputCSV3()

# 建立預測資料

In [ ]:
df = pd.read_csv('data_with_predict_price.csv')

In [ ]:
df.shape

In [ ]:
df['sqft'] = pd.to_numeric(df['sqft']).astype('int64')

In [ ]:
cols = ['zipcode','bedroom','bathroom','sqft','houseprice','predictPrice']
zipcode = [98010, 98029, 98092, 98075 ,98106]
bedroom = [2,3,4]
bathroom = [2,3,4]
sqft = [1000,2000,3000,4000]

In [ ]:
df1 = df[cols]

In [ ]:
df1[(df1['zipcode']==98029)&(df1['houseprice']>1000000)&(df1['houseprice']<1800000)]

In [ ]:
pre_list=[]
for i in zipcode:
    for j in bedroom:
        for k in bathroom:
            for l in sqft:
                df2 = df1[(df1['zipcode']==i)&(df1['bedroom']==j)&(df1['bathroom']>=k)&(df1['bathroom']<k+1)&(df1['sqft']>=l)&(df1['sqft']<l+1000)]
                if df2.empty:
                    print(i,j,k,l)
                else:
                    l1 = str(l)+'~'+str(l+1000)
                    df3 = df2.groupby('zipcode').apply(sum)
                    pre_list.append([i,j,k,l1,round(df3.iloc[0,4]/df3.iloc[0,3],1),round(df3.iloc[0,5]/df3.iloc[0,3],1)]) #1000 取代 round(df3.iloc[0,5]/df3.iloc[0,3],1)

In [ ]:
pre_list

In [ ]:
for list1 in pre_list:
    if ((list1[4]-list1[5]) < 200) &((list1[4]-list1[5]) > -200):
        print(list1)

In [ ]:
def OutputCSV4():
    cols = ['zipcode','bedroom','bathroom','sqft','Predict_price_sqft','Actual_price_sqft']
    file_name = "pre_price.csv"
    pd.DataFrame(pre_list, columns = cols).to_csv(file_name, index = False, encoding = "UTF-8")

In [ ]:
OutputCSV4()

# 建立熱區資料csv

In [ ]:
df = pd.read_csv('data_with_predict_price.csv')

In [ ]:
df.shape

In [ ]:
dfzipcode = df['zipcode']

In [ ]:
dfzipcode = dfzipcode.drop_duplicates()

In [ ]:
dfzipcode

In [ ]:
zipcodelist = []
for j in range(len(dfzipcode)):
    zipcodelist += [dfzipcode.iloc[j]]

In [ ]:
zipcodelist = sorted(zipcodelist)

In [ ]:
zipcodelist

In [ ]:
df2 = df[['zipcode','sqft','predictPrice']]

In [ ]:
df2

In [ ]:
df2['sqft'] = pd.to_numeric(df2['sqft']).astype('int64')

In [ ]:
df3 = df2.groupby('zipcode').apply(sum)

In [ ]:
df3

In [ ]:
hotprice = []

for i in range(len(zipcodelist)):
    df3.iloc[i,2]/df3.iloc[i,1]
    hotprice.append([int(zipcodelist[i]),round(df3.iloc[i,2]/df3.iloc[i,1])])

In [ ]:
hotprice

In [ ]:
hotprice_d = dict(hotprice)

In [ ]:
hotprice_d = str(hotprice_d)

In [ ]:
for i in zipcodelist:
    i = int(i)
    hotprice_d = hotprice_d.replace(f'{i}',f'"{i}"')

In [ ]:
hotprice_d

In [ ]:
def OutputCSV5():
    file_name = "hotprice.csv"
    columns = ['zipcode','Actual_price_sqft']
    pd.DataFrame(hotprice, columns = columns).to_csv(file_name, index = False, encoding = "UTF-8")

In [ ]:
OutputCSV5()

# 處理重複地址

In [ ]:
df = pd.read_csv('filename.csv')

In [ ]:
df['address']

In [ ]:
for i in range(len(df['address'])):
    df['address'][i] = df['address'][i].replace(',-','-')

In [ ]:
address_1 = []
for i in range(len(df)):
    address_1 += [df['address'][i]]
#print(df1['html'][i][:-15][35:])
print(address_1)

In [ ]:
df1 = pd.read_csv('filename.csv')

In [ ]:
for i in range(len(df1)):
    if df1['html'][i][:-15][35:] in address_1:
        df1['html'][i] = 0
    elif df1['html'][i][:-16][35:] in address_1:
        df1['html'][i] = 0
    elif df1['html'][i][:-17][35:] in address_1:
        df1['html'][i] = 0
    else :
        print('none')

In [ ]:
df2 = df1.drop_duplicates()

In [ ]:
df2

In [ ]:
def OutputCSV6():       
    Result ='filename.csv'
    df_SAMPLE = pd.DataFrame.from_dict(house)
    df_SAMPLE.to_csv(df2, index=False)
    print('Finish')

In [ ]:
OutputCSV6()

# 資料清洗

In [ ]:
import pandas as pd
import numpy as np

house = pd.read_csv('98112.csv')

In [ ]:
house.shape

In [ ]:
len_5 = len(house['school3'])
for i in range(len_5):
    if pd.isnull(house['school3'][i]) is True:
        house['school3'][i] = 0
#        house['school2'][i] = 5

In [ ]:
house['school3'].isnull().any()

In [ ]:
len_5 = len(house['school3'])
count=0
for i in range(len_5):
    if house['school2'][i] == 1 and pd.isnull(house['school3'][i]) is True:
        house['school3'][i] = house['school2'][i]
        count+=1
print(count)
        #house['school3'][i] = house['school2'][i]
#        house['school2'][i] = 5

In [ ]:
len_5 = len(house['school3'])
for i in range(len_5):
    if house['school2'][i] == 6 and pd.isnull(house['school3'][i]) is True:
        count+=1
        house['school3'][i] = house['school2'][i]
        house['school2'][i] = house['school1'][i]
print(count)
        #house['school3'][i] = house['school2'][i]
#        house['school2'][i] = 5

In [ ]:
house['school3'].isnull().any()

In [ ]:
len(house['lot'])

In [ ]:
# lot處理單位轉換acres to sqft
len_1 = len(house['lot'])
for i in range(len_1):
    try:
        if house['lot'][i][-1] == 's': 
            house['lot'][i] = int(house['lot'][i].replace(' Acres','').replace('.',''))*(435) # 要轉int忽略小數後455.60 ???
        elif house['lot'][i][-1] == 't': 
            house['lot'][i] = int(house['lot'][i].replace(' sqft','').replace(',',''))
        else:
            house['lot'][i] = int(house['lot'][i])
    except:
        print(i,house['lot'][i])

In [ ]:
# object轉換成int, 並將nan補0, coerce將為無法轉為int轉為nan
house['lot'] = pd.to_numeric(house['lot'], errors='coerce').fillna('0').astype('int64')

In [ ]:
# 確認空值, False
house['lot'].isnull().any()

In [ ]:
house['lot']

In [ ]:
house['houseyear']

In [ ]:
# 確認空值, False
house['houseyear'].isnull().any()

In [ ]:
# houseyear數據整理
len_2 = len(house['houseyear'])
for i in range(len_2):
    if house['houseyear'][i] == 'No Data':
        house['houseyear'][i] == np.nan
        print(i)

In [ ]:
house['houseyear'][266]

In [ ]:
med_houseyear = int(house['houseyear'].median())
med_houseyear

In [ ]:
house['houseyear'] = house['houseyear'].fillna(med_houseyear)

In [ ]:
house['houseyear'].isnull().any()

In [ ]:
# houseyear object轉換成int, 並將nan補0, coerce將為無法轉為int轉為nan
house['houseyear'] = pd.to_numeric(house['houseyear']).astype('int64')

In [ ]:
house['houseyear']

In [ ]:
mean_walkscore = int(house['walkscore'].mean())
mean_walkscore

In [ ]:
house['walkscore'] = house['walkscore'].fillna(mean_walkscore)

In [ ]:
house['walkscore'].isnull().any()

In [ ]:
house['walkscore'] = pd.to_numeric(house['walkscore']).astype('int64')

In [ ]:
house['walkscore']

In [ ]:
mean_transitscore = int(house['transitscore'].mean())
mean_transitscore

In [ ]:
house['transitscore'] = house['transitscore'].fillna(0)

In [ ]:
house['transitscore'].isnull().any()

In [ ]:
house['transitscore'] = pd.to_numeric(house['transitscore']).astype('int64')

In [ ]:
house['transitscore']

In [ ]:
mean_houseprice = int(house['houseprice'].mean())
mean_houseprice

In [ ]:
mean_pt = int(house['pt'].mean())
mean_pt

In [ ]:
mean_ta = int(house['ta'].mean())
mean_ta

In [ ]:
len_3 = len(house['pt'])
for i in range(len_3):
    if pd.isnull(house['pt'][i]) is True:
        house['pt'][i] = mean_pt/mean_houseprice*house['houseprice'][i]

In [ ]:
house['pt'].isnull().any()

In [ ]:
house['pt'] = pd.to_numeric(house['pt']).astype('int64')

In [ ]:
house['pt']

In [ ]:
len_4 = len(house['ta'])
for i in range(len_3):
    if pd.isnull(house['ta'][i]) is True:
        house['ta'][i] = mean_ta/mean_houseprice*house['houseprice'][i]

In [ ]:
house['ta'].isnull().any()

In [ ]:
house['ta'] = pd.to_numeric(house['ta']).astype('int64')

In [ ]:
house['ta']

# 資料合併

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def read_inf(data):
    df1 = pd.read_csv(data+'.csv')
    df2 = pd.read_csv('zipcodeData.csv')    # 房屋特徵(中文)
    df3 = pd.read_csv('zipcodeDataUs.csv')  # 房屋特徵(英文)
    return (df1,df2,df3)

In [ ]:
def add_columns(df1,df2,df3,counts):
    # zipcode刪除? 順便將地址轉換成zipcode?
    columns1=['zipcode', '當前人口', '男性人口', '女性人口', '性別比', '白人人口', '黑人人口', '西班牙裔人口', '亞洲人口', '夏威夷人口', '印度裔人口', '其他人口',  '白人比率', '黑人比率', '西班牙裔比率', '亞洲人比率', '夏威夷人比率', '印度裔人比率', '其他人比率', '年齡的中位數', '男性年齡中位數', '女性年齡中位數', '每戶人數', '平均房屋價值', '每戶收入']
    columns2=['zipcode', 'population Density', 'housing Units', 'land Area', 'median Household Income', 'households With Kids',' built before 1970', ' built after 1970','Car', 'High School', "Associate's degree", "Bachelor's degree","Master's degree", '<25k', '25k-60k', '60k-150k', '150k+','<25krate', '25k-60k rate', '60k-150k rate', '150k+ rate']
    for i in range(len(columns1)):
        df1.insert(17+i, column=columns1[i], value=[df2[columns1[i]][counts]]*len(df1['houseyear']))
    df1 = df1.drop("zipcode", axis = 1)
    for i in range(len(columns2)):
        df1.insert(17+len(columns1)-1+i, column=columns2[i], value=[df3[columns2[i]][counts]]*len(df1['houseyear']))
    df1 = df1.drop("zipcode", axis = 1)
    return df1

In [ ]:
def OutputCSV7(data,df1):         
    Result = data+'_1.csv'
    df_SAMPLE = pd.DataFrame.from_dict(df1)
    df_SAMPLE.to_csv(Result, index=False)   
    print(data,':Finish')

#### 執行前更改zipcode

In [ ]:
# 更改成自己的zipcode, 提醒這裡的順序需跟房屋特徵順序要一樣!!!!
zipcodes = ['98006', '98007', '98024', '98115', '98092', '98198', '98072', '98010', '98199', '98146', '98112', '98065', '98107', '98188', '98148', '98121', '98051']

counts = 0 
for data in zipcodes:  
    read_inf(data)
    df1 = add_columns(read_inf(data)[0],read_inf(data)[1],read_inf(data)[2],counts)
    counts +=1
    OutputCSV7(data,df1)

# 測試

In [ ]:
df['date'][0]

In [ ]:
parse(df['date'][0]).year

In [ ]:
parse(df['date'][0]).month

In [ ]:
parse(df['date'][0]).day